In [1]:
import pandas as pd
from utils import function
import numpy as np
from scipy import stats

In [2]:
def transformar_em_dummy(dataframe, coluna):
    # Iterar sobre as linhas do DataFrame
    categorias=dataframe[coluna].unique()
    for indice, linha in dataframe.iterrows():
        # Iterar sobre os valores na linha
        for item in categorias:
            if linha[coluna]==item:
                dataframe[f"{coluna}_{item}"]=1
            else:
                dataframe[f"{coluna}_{item}"]=0
            # Criar uma nova coluna com o nome do valor e atribuir 1 se for igual ao valor atual, 0 caso contrário
    
    # Preencher valores NaN com 0
    #dataframe.fillna(0, inplace=True)
    
    return dataframe

In [3]:
def calcular_correlacao(df, coluna1, coluna2):
    df_filtrado = df.dropna(subset=[coluna1, coluna2])
    corr=df_filtrado[coluna1].corr(df_filtrado[coluna2])
    valor_p = stats.pearsonr(df_filtrado[coluna1], df_filtrado[coluna2])[1]
    return corr,valor_p

In [4]:
def igualar_tamanho(df1, df2):
    if len(df1) > len(df2):
        df1=df1.head(-1)
    elif len(df2) > len(df1):
        df2=df2.head(-1)
    else:
        a=1
    return df1, df2

In [5]:
df=pd.read_excel("/home/lucas/Documentos/SentimentAnalysisInMentalHealth/Datasets/ResultadoFineTuning.xlsx")

In [6]:
df_output=pd.DataFrame(columns=['sessao', 
                                'sentimento paciente',
                                'itens_paciente', 
                                'sentimento terapeuta',
                                'itens_terapeuta',
                                'window',
                                'winc',
                                'lag',
                                'max_mean',
                                'min_mean',
                                'max_median',
                                'min_median',
                                'WaiPaciente',
                                'WaiTerapeuta'])

for sessao in df.sessao.unique():
    df_sessao=df.loc[df["sessao"]== sessao]
    sentimentos=["score_fine_tuning_neutro","score_fine_tuning_positivo","score_fine_tuning_negativo"]
    combinacoes_sentimento = [(sentimento_p, sentimento_t) for sentimento_p in ["score_fine_tuning_neutro","score_fine_tuning_positivo","score_fine_tuning_negativo"] 
                                       for sentimento_t in ["score_fine_tuning_neutro","score_fine_tuning_positivo","score_fine_tuning_negativo"] ]
        
        # Loop sobre as combinações de itens_paciente e itens_terapeuta
    for sentimento_paciente, sentimento_terapeuta in combinacoes_sentimento:
        sumarizacao_sessao_paciente=function.processar_dataframe_float(df_sessao,"Falante",sentimento_paciente)
        sumarizacao_sessao_terapeuta=function.processar_dataframe_float(df_sessao,"Falante",sentimento_terapeuta)
        df_paciente=sumarizacao_sessao_paciente[sumarizacao_sessao_paciente["Categoria"]=="p"]
        df_terapeuta=sumarizacao_sessao_terapeuta[sumarizacao_sessao_terapeuta["Categoria"]=="t"]
        df_paciente,df_terapeuta=igualar_tamanho(df_paciente,df_terapeuta)
        combinacoes_itens = [(p, t) for p in ["Primeiro_valor_B", "Ultimo_valor_B", "Media_B"] 
                                       for t in ["Primeiro_valor_B", "Ultimo_valor_B", "Media_B"]]
        
        # Loop sobre as combinações de itens_paciente e itens_terapeuta
        for itens_paciente, itens_terapeuta in combinacoes_itens:
            dataframe_serie = pd.DataFrame(columns=['Paciente', 'Terapeuta'])
            dataframe_serie['Paciente']=df_paciente[itens_paciente].values
            dataframe_serie['Terapeuta']=df_terapeuta[itens_terapeuta].values
            for window,winc,lag in [(15,4,4),(10,4,4),(20,4,4),(10,2,2),(6,2,2)]:
                    df_wcc=function.windowed_cross_correlation(dataframe_serie, 
                                                               "Paciente", 
                                                               "Terapeuta", 
                                                               window_size=window,
                                                               winc=winc,
                                                               lag=lag)
                    df_output = pd.concat([df_output,pd.DataFrame.from_dict({'sessao': [sessao], 
                                                                             'sentimento paciente':[sentimento_paciente],
                                                                             'itens_paciente': [itens_paciente], 
                                                                             'sentimento terapeuta':[sentimento_terapeuta],
                                                                             'itens_terapeuta': [itens_terapeuta], 
                                                                             'window':[window],
                                                                             'winc':[winc],
                                                                             'lag':[lag],
                                                                             'max_mean':[df_wcc.max_value.mean()],
                                                                             'min_mean':[df_wcc.min_value.mean()],
                                                                             'max_median':[df_wcc.max_value.median()],
                                                                             'min_median':[df_wcc.min_value.median()],
                                                                             'WaiPaciente':[df_sessao["WAI-SR-P TOTAL"].values[0]],
                                                                             'WaiTerapeuta':[df_sessao["WAI-SR-T TOTAL"].values[0]]
                                                                            })])

/home/lucas/Documentos/SentimentAnalysisInMentalHealth/utils/function.py:139: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  novo_dataframe = pd.concat([novo_dataframe,pd.DataFrame.from_dict({'Categoria': [categoria_anterior],
/home/lucas/Documentos/SentimentAnalysisInMentalHealth/utils/function.py:139: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  novo_dataframe = pd.concat([novo_dataframe,pd.DataFrame.from_dict({'Categoria': [categoria_anterior],
/tmp/ipykernel_8123/1714376385.py:44: FutureWarn

In [7]:
#df[['A', 'B']]
df_output_unique=df_output[["sentimento paciente",
           "itens_paciente",
           "sentimento terapeuta","itens_terapeuta",	
           "window",
           "winc",
           "lag"]].drop_duplicates()

In [8]:
df_output_unique.shape

(405, 7)

In [9]:
correl_max_mean_list_paciente=[]
p_valor_max_mean_list_paciente=[]
correl_max_mean_list_terapeuta=[]
p_valor_max_mean_list_terapeuta=[]
correl_min_mean_list_paciente=[]
p_valor_min_mean_list_paciente=[]
correl_min_mean_list_terapeuta=[]
p_valor_min_mean_list_terapeuta=[]
correl_max_median_list_paciente=[]
p_valor_max_median_list_paciente=[]
correl_max_median_list_terapeuta=[]
p_valor_max_median_list_terapeuta=[]
correl_min_median_list_paciente=[]
p_valor_min_median_list_paciente=[]
correl_min_median_list_terapeuta=[]
p_valor_min_median_list_terapeuta=[]
for index,row in df_output_unique.iterrows():
    df_result_comb=df_output[(df_output["sentimento paciente"]==row["sentimento paciente"])&
                            (df_output["itens_paciente"]==row["itens_paciente"])&
                            (df_output["sentimento terapeuta"]==row["sentimento terapeuta"])&
                            (df_output["itens_terapeuta"]==row["itens_terapeuta"])&
                            (df_output["window"]==row["window"])&
                            (df_output["winc"]==row["winc"])&
                            (df_output["lag"]==row["lag"])
                            ]
    correl_max_mean_list_terapeuta.append(calcular_correlacao(df_result_comb, 'max_mean', 'WaiTerapeuta')[0])
    p_valor_max_mean_list_terapeuta.append(calcular_correlacao(df_result_comb, 'max_mean', 'WaiTerapeuta')[1])
    correl_max_mean_list_paciente.append(calcular_correlacao(df_result_comb, 'max_mean', 'WaiPaciente')[0])
    p_valor_max_mean_list_paciente.append(calcular_correlacao(df_result_comb, 'max_mean', 'WaiPaciente')[1])
    correl_min_mean_list_terapeuta.append(calcular_correlacao(df_result_comb, 'min_mean', 'WaiTerapeuta')[0])
    p_valor_min_mean_list_terapeuta.append(calcular_correlacao(df_result_comb, 'min_mean', 'WaiTerapeuta')[1])
    correl_min_mean_list_paciente.append(calcular_correlacao(df_result_comb, 'min_mean', 'WaiPaciente')[0])
    p_valor_min_mean_list_paciente.append(calcular_correlacao(df_result_comb, 'min_mean', 'WaiPaciente')[1])
    correl_max_median_list_terapeuta.append(calcular_correlacao(df_result_comb, 'max_median', 'WaiTerapeuta')[0])
    p_valor_max_median_list_terapeuta.append(calcular_correlacao(df_result_comb, 'max_median', 'WaiTerapeuta')[1])
    correl_max_median_list_paciente.append(calcular_correlacao(df_result_comb, 'max_median', 'WaiPaciente')[0])
    p_valor_max_median_list_paciente.append(calcular_correlacao(df_result_comb, 'max_median', 'WaiPaciente')[1])
    correl_min_median_list_terapeuta.append(calcular_correlacao(df_result_comb, 'min_median', 'WaiTerapeuta')[0])
    p_valor_min_median_list_terapeuta.append(calcular_correlacao(df_result_comb, 'min_median', 'WaiTerapeuta')[1])
    correl_min_median_list_paciente.append(calcular_correlacao(df_result_comb, 'min_median', 'WaiPaciente')[0])
    p_valor_min_median_list_paciente.append(calcular_correlacao(df_result_comb, 'min_median', 'WaiPaciente')[1])
    #print(comb,df_filtrado_max_mean_T['max_mean'].corr(df_filtrado_max_mean_T['WaiTerapeuta']))

In [10]:
df_output_unique["correl_max_mean_list_terapeuta"]=correl_max_mean_list_terapeuta
df_output_unique["p_valor_max_mean_list_terapeuta"]=p_valor_max_mean_list_terapeuta
df_output_unique["correl_max_mean_list_paciente"]=correl_max_mean_list_paciente
df_output_unique["p_valor_max_mean_list_paciente"]=p_valor_max_mean_list_paciente
df_output_unique["correl_min_mean_list_terapeuta"]=correl_min_mean_list_terapeuta
df_output_unique["p_valor_min_mean_list_terapeuta"]=p_valor_min_mean_list_terapeuta
df_output_unique["correl_min_mean_list_paciente"]=correl_min_mean_list_paciente
df_output_unique["p_valor_min_mean_list_paciente"]=p_valor_min_mean_list_paciente
df_output_unique["correl_max_median_list_terapeuta"]=correl_max_median_list_terapeuta
df_output_unique["p_valor_max_median_list_terapeuta"]=p_valor_max_median_list_terapeuta
df_output_unique["correl_max_median_list_paciente"]=correl_max_median_list_paciente
df_output_unique["p_valor_max_median_list_paciente"]=p_valor_max_median_list_paciente
df_output_unique["correl_min_median_list_terapeuta"]=correl_min_median_list_terapeuta
df_output_unique["p_valor_min_median_list_terapeuta"]=p_valor_min_median_list_terapeuta
df_output_unique["correl_min_median_list_paciente"]=correl_min_median_list_paciente
df_output_unique["p_valor_min_median_list_paciente"]=p_valor_min_median_list_paciente

In [11]:
df_output_unique.to_excel("resultado_WCC_final_BERT.xlsx")

In [ ]:
df_wcc